# Проект для «Викишоп»

**Краткая суть**

Необходимо обучить модель классифицировать комментарии на позитивные и негативные

**Описание**

Интернет-магазин «Викишоп» запускает новый сервис, где пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.


**Критерии оценки модели**

* Значение метрики F1 на тестовой выборке должно быть не меньше 0.75

**План выполнения проекта**

1. Загрузить и подготовьть данные.
2. Обучить разные модели.
3. Сделать выводы.

## Подготовка

### Загрузим необходимые библиотеки

In [1]:
import warnings
warnings.filterwarnings('ignore')

import time

import pandas as pd
import numpy as np

from sklearn.utils import shuffle

import optuna

import os

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.dummy import DummyClassifier
from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, classification_report, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

import re 

### Посмотрим на данные

In [2]:
virtual = '/datasets/toxic_comments.csv'
local = '/file_for_projects/toxic_comments.csv'

if os.path.exists(virtual):
    df = pd.read_csv(virtual)
elif os.path.exists(local):
    df = pd.read_csv(local)
else:
    print('Something is wrong')

In [3]:
def check_data(data):
    print('')
    print('')
    print('Информация о файле')
    print('')
    df.info()
    print('')
    print('')
    print('Первые 5 строк файла')
    display(df.head())
    print('')
    print('')
    print('Количество пропусков')
    display(df.isna().sum())
    print('')
    print('')
    print('Количество дубликатов')
    display(df.index.duplicated().sum())
    print('')

In [4]:
check_data(df)



Информация о файле

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Первые 5 строк файла


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0




Количество пропусков


Unnamed: 0    0
text          0
toxic         0
dtype: int64



Количество дубликатов


0

**У нас есть датафрейм в котором 159292 строки и 3 столбца. В столбце text содержится комментарий (на английском языке), столбец toxic целевой, в нем 0 означает "комментарий не токсичный", а 1 говорит о том, что комментарий токсичный. От стообца Unnamed избавимся.**

In [5]:
df = df.drop('Unnamed: 0', axis=1)

### Посмотрим на распределение комментариев

In [6]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

**Мы наблюдаем дисбаланс классв, не токсичных в 10 раз больше. Посмотрим какими получатся метрики, и если значение не будет нас удовлетворять, будем решать что делать с этим.**

### Посмотрим качество разметки

In [7]:
for i in df[df['toxic'] == 0]['text'].sample(10, random_state=22):
    print (i)

I said that it's a mix of the various bits, anyway, I'm sure we'll email back and fourth a few times before I get permission, so I'll tell then if they specifically ask. However, I feel that it's still fair use. (talk|email)
"

 NintendoLife Retrospective 

 http://www.nintendolife.com/news/2015/06/matters_of_import_fire_emblem_sort_of_exists_on_the_sony_playstation 

While I can't help but notice that they decided to do a retrospective just a little after I've completely rewritten the article (and cover most of the same points), but it can still offer some new info to the article, mostly reception-related content.  msg me "
Invitation to a Wicnic in Gainesville on Saturday, June 22nd 

Greetings!

Seeing that you're a member of WikiProject University of Florida, I'm inviting to the North Central Florida 2013 Great American Wiknic that will be on Saturday June 22, 2013, commencing at 1:00 pm, ten blocks north of UF campus in Gainesville,.

If you're able and inclined to come, please RS

In [8]:
for i in df[df['toxic'] == 1]['text'].sample(10, random_state=22):
    print (i)

Wikipedia for online losers

I am being accused of vandalizing Afghanistan's article....because the accusers are all jealous of me hahahahahahahaha.  09:32, 31 October 2006
Knock off the gosh darn vandalism! 

If you keep on vandalising articles as you did to Poop I'm going to warn your ass.  This is a federal offense and you know what happens in prison.  ;)  You better watch yourself or I will unleash a powerful series of warnings.  I've got like a buttload of respect around here and I don't take kindly to people who come to these places and treat it like SHIT!  DX
Jared Loughner and the Saphir-Wolf hypothesis 

Dude. Loughner said that vocabulary was invented to control people's minds. That's exactly what the Saphir-Wolf hypothesis says. Don't call me libelous, irrelevant and unsourced. Jerk.
stop being a pussbot 

stop being a pussbot. you are extremely biased. Wikipedia cannto tolerate users like you. stop being a shudslavpus
Hitlers sex children 

Its obvious that shitler only wan

**По представленным примерам, разметка кажется хорошей. У нас все равно нет вариантов, т к разметка производилась не нами, будем верить тому, что есть.**

In [9]:
train, test = train_test_split(df, test_size=0.3, random_state=22)

In [10]:
valid, test = train_test_split(test, test_size=0.29, random_state=22)

In [11]:
train.shape, valid.shape, test.shape

((111504, 2), (33929, 2), (13859, 2))

In [12]:
print(f'Распределение комментариев на обучающей выборке', train['toxic'].value_counts())
print(f'Распределение комментариев на валидационной выборке', valid['toxic'].value_counts())
print(f'Распределение комментариев на тестовой выборке', test['toxic'].value_counts())

Распределение комментариев на обучающей выборке 0    100104
1     11400
Name: toxic, dtype: int64
Распределение комментариев на валидационной выборке 0    30553
1     3376
Name: toxic, dtype: int64
Распределение комментариев на тестовой выборке 0    12449
1     1410
Name: toxic, dtype: int64


**Закономерность "в 10 раз" сохранена**

### Проивзедем предобработку данных

#### Напишем функции для преобразования данных. На вход будем подавать текст, а получать токены (без окончаний, стов-слов и знаков препинания)

In [1]:
eng_stopwords = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

def change_word(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language='english') # разбиваем текст на токены
    if remove_stop_words:
        tokens = [i for i in tokens if i not in eng_stopwords] # удаляем стоп-слова
    tokens = [lemmatizer.lemmatize(i) for i in tokens] # лемматизируем токены
    return tokens


def clear_text(sentence: str, remove_stop_words: bool = True):  # функция для удаления знаков препинания
    sentence = re.sub(r'[^a-zA-Z]', ' ', sentence)
    sentence = sentence.split()
    return " ".join(sentence)

#### Проверим что всё работает

In [14]:
df.iloc[202]['text']

'"\nReliable sources indicate otherwise. Please do not insert your personal analysis into the article.  (talk) "'

In [15]:
change_word(clear_text(df.iloc[202]['text']))

['Reliable',
 'source',
 'indicate',
 'otherwise',
 'Please',
 'insert',
 'personal',
 'analysis',
 'article',
 'talk']

#### Инициализируем TF-IFD векторайзер, который подготовит из текста числовую матрицу для модели

In [16]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: change_word(clear_text(x, remove_stop_words=True)))

## Обучение

### LogisticRegression

In [17]:
lr_pipelene = Pipeline([
    ("vectorizer", vectorizer),
    ("model", LogisticRegression(random_state=22, solver='lbfgs'))
])

In [18]:
lr_pipelene.fit(train['text'], train['toxic'])

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x000002721FB15310>)),
                ('model', LogisticRegression(random_state=22))])

In [19]:
lr = lr_pipelene.fit(train['text'], train['toxic'])

In [20]:
lr_pred = lr.predict(valid['text'])

In [21]:
lr_f1 = f1_score(valid['toxic'], lr_pred)

In [22]:
print(classification_report(valid['toxic'], lr_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98     30553
           1       0.92      0.62      0.74      3376

    accuracy                           0.96     33929
   macro avg       0.94      0.80      0.86     33929
weighted avg       0.96      0.96      0.95     33929



In [23]:
cm = confusion_matrix(valid['toxic'], lr_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[30378   175]
 [ 1299  2077]]

True Positives(TP) =  30378

True Negatives(TN) =  2077

False Positives(FP) =  175

False Negatives(FN) =  1299


**Логистическая регрессия, без подбора гиперпараметров, может достаточно хорошо делить комментарии, но страдает точность. Из матрици ошибок следует, что 1299 ответов ложноотрицательные и 175 ложноположительных** 

### SGDClassifier

In [24]:
sgdc_pipelene = Pipeline([
    ("vectorizer", vectorizer),
    ("model", SGDClassifier(random_state=22))
])

In [25]:
sgdc = sgdc_pipelene.fit(train['text'], train['toxic'])

In [26]:
sgdc_pred = sgdc.predict(valid['text'])

In [27]:
sgdc_f1 = f1_score(valid['toxic'], sgdc_pred)

In [28]:
print(classification_report(valid['toxic'], sgdc_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97     30553
           1       0.96      0.48      0.64      3376

    accuracy                           0.95     33929
   macro avg       0.95      0.74      0.81     33929
weighted avg       0.95      0.95      0.94     33929



In [29]:
cm = confusion_matrix(valid['toxic'], sgdc_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[30485    68]
 [ 1750  1626]]

True Positives(TP) =  30485

True Negatives(TN) =  1626

False Positives(FP) =  68

False Negatives(FN) =  1750


**Модель SGDClassifier (с простой реализацией обучения стохастическим градиентным спуском) , без подбора гиперпараметров, может достаточно хорошо делить комментарии, но точность страдает еще больше. Из матрици ошибок следует, что 1750 ответов ложноотрицательные и 68 ложноположительные** 

### LGBMClassifier

In [30]:
lgb_pipelene = Pipeline([
    ("vectorizer", vectorizer),
    ("model", LGBMClassifier(random_state=22))
])

In [31]:
lgb = lgb_pipelene.fit(train['text'], train['toxic'])

In [32]:
lgb_pred = lgb.predict(valid['text'])

In [33]:
lgb_f1 = f1_score(valid['toxic'], lgb_pred)

In [34]:
print(classification_report(valid['toxic'], lgb_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98     30553
           1       0.89      0.65      0.75      3376

    accuracy                           0.96     33929
   macro avg       0.92      0.82      0.86     33929
weighted avg       0.95      0.96      0.95     33929



In [35]:
cm = confusion_matrix(valid['toxic'], lgb_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[30273   280]
 [ 1180  2196]]

True Positives(TP) =  30273

True Negatives(TN) =  2196

False Positives(FP) =  280

False Negatives(FN) =  1180


**Модель LGBMClassifier (на градиентном бустинге) , без подбора гиперпараметров, тоже хорошо себя демострирует, но вот точность чуть лучше, чем у предыдущих двух моделей. Из матрици ошибок следует, что 1180 ответов ложноотрицательные и 280 ложноположительные** 

### Побдерем гиперпараметры для LGBMClassifier

In [36]:
def objective(trial):
    classifier = Pipeline(steps=[("vectorizer", vectorizer),
                                 ("classifier",LGBMClassifier(random_state=22))])

    params = {
        'classifier__lambda_l1': trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        'classifier__lambda_l2': trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        'classifier__num_leaves': trial.suggest_int("num_leaves", 2, 256),
        'classifier__feature_fraction': trial.suggest_float("feature_fraction", 0.4, 1.0),
        'classifier__bagging_fraction': trial.suggest_float("bagging_fraction", 0.4, 1.0),
        'classifier__bagging_freq': trial.suggest_int("bagging_freq", 1, 7),
        'classifier__min_child_samples': trial.suggest_int("min_child_samples", 5, 100)
    }

    classifier.set_params(**params)

    classifier.fit(train['text'], train['toxic'])

    f1 = cross_val_score(classifier, valid['text'], valid['toxic'], cv=5, scoring='f1')

    return  f1.mean()

In [37]:
%%time
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

[I 2022-12-20 21:01:43,126] A new study created in memory with name: no-name-7527449e-513b-4d2c-a356-8019f439f728


[LightGBM] [Warning] feature_fraction is set=0.6483993578956935, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6483993578956935
[LightGBM] [Warning] bagging_fraction is set=0.6404334844550104, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6404334844550104
[LightGBM] [Warning] lambda_l1 is set=1.7747473273573157e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.7747473273573157e-06
[LightGBM] [Warning] lambda_l2 is set=1.8348746771410183e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8348746771410183e-08
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.6483993578956935, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6483993578956935
[LightGBM] [Warning] bagging_fraction is set=0.6404334844550104, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6404334844550104
[LightG

[I 2022-12-20 21:05:16,051] Trial 0 finished with value: 0.6569975088769185 and parameters: {'lambda_l1': 1.7747473273573157e-06, 'lambda_l2': 1.8348746771410183e-08, 'num_leaves': 133, 'feature_fraction': 0.6483993578956935, 'bagging_fraction': 0.6404334844550104, 'bagging_freq': 4, 'min_child_samples': 84}. Best is trial 0 with value: 0.6569975088769185.


[LightGBM] [Warning] feature_fraction is set=0.889564291608139, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.889564291608139
[LightGBM] [Warning] bagging_fraction is set=0.43355649835118754, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43355649835118754
[LightGBM] [Warning] lambda_l1 is set=1.5500227471617577e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5500227471617577e-07
[LightGBM] [Warning] lambda_l2 is set=0.018433780898542865, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.018433780898542865
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.889564291608139, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.889564291608139
[LightGBM] [Warning] bagging_fraction is set=0.43355649835118754, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43355649835118754
[LightGBM] 

[I 2022-12-20 21:08:28,963] Trial 1 finished with value: 0.680352713188585 and parameters: {'lambda_l1': 1.5500227471617577e-07, 'lambda_l2': 0.018433780898542865, 'num_leaves': 91, 'feature_fraction': 0.889564291608139, 'bagging_fraction': 0.43355649835118754, 'bagging_freq': 3, 'min_child_samples': 64}. Best is trial 1 with value: 0.680352713188585.


[LightGBM] [Warning] feature_fraction is set=0.7287594166181555, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7287594166181555
[LightGBM] [Warning] bagging_fraction is set=0.7517604150760278, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7517604150760278
[LightGBM] [Warning] lambda_l1 is set=0.00018720624497429878, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00018720624497429878
[LightGBM] [Warning] lambda_l2 is set=0.40883920798727896, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.40883920798727896
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7287594166181555, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7287594166181555
[LightGBM] [Warning] bagging_fraction is set=0.7517604150760278, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7517604150760278
[LightGBM] [W

[I 2022-12-20 21:12:02,417] Trial 2 finished with value: 0.6697023989778538 and parameters: {'lambda_l1': 0.00018720624497429878, 'lambda_l2': 0.40883920798727896, 'num_leaves': 221, 'feature_fraction': 0.7287594166181555, 'bagging_fraction': 0.7517604150760278, 'bagging_freq': 3, 'min_child_samples': 78}. Best is trial 1 with value: 0.680352713188585.


CPU times: total: 17min 37s
Wall time: 10min 19s


In [38]:
print(study.best_trial.params)
print(study.best_value)

{'lambda_l1': 1.5500227471617577e-07, 'lambda_l2': 0.018433780898542865, 'num_leaves': 91, 'feature_fraction': 0.889564291608139, 'bagging_fraction': 0.43355649835118754, 'bagging_freq': 3, 'min_child_samples': 64}
0.680352713188585


In [40]:
features_train = vectorizer.fit_transform(train['text'])
features_valid= vectorizer.transform(valid['text'])
features_test= vectorizer.transform(test['text'])

In [41]:
%%time
model = LGBMClassifier(**study.best_params, random_state=22).fit(features_train, train['toxic'])

[LightGBM] [Warning] feature_fraction is set=0.889564291608139, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.889564291608139
[LightGBM] [Warning] bagging_fraction is set=0.43355649835118754, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43355649835118754
[LightGBM] [Warning] lambda_l1 is set=1.5500227471617577e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5500227471617577e-07
[LightGBM] [Warning] lambda_l2 is set=0.018433780898542865, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.018433780898542865
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
CPU times: total: 1min 37s
Wall time: 26.4 s


In [42]:
pred = model.predict(features_valid)

In [43]:
print('F1 на валидационной выборке:', np.round(f1_score(valid['toxic'], pred), 4))

F1 на валидационной выборке: 0.7603


In [44]:
pred = model.predict(features_test)

In [45]:
print('F1 на тестовой выборке:', np.round(f1_score(test['toxic'], pred), 4))

F1 на тестовой выборке: 0.7732


## Выводы

In [46]:
result = pd.DataFrame({'model' : ['LogisticRegression', 
                                  'SGDClassifier', 
                                  'LGBMClassifier' 
                                 ],
                       'F1 дефолтной модели' : [round(lr_f1, 4) , 
                                                round(sgdc_f1, 4),
                                                round(lgb_f1, 4)
                                               ]
                      })
result

,model,F1 дефолтной модели
0,LogisticRegression,0.7381
1,SGDClassifier,0.6414
2,LGBMClassifier,0.7505


***Из таблицы видно, что без подбора параметров, лучше всех на валидационой выборке себя показывает LGBMClassifier. Для модели были подобраны гиперпараметры и произведена проверка на тесте***

In [47]:
print('F1 LGBMClassifier c подобранными гиперпараметрами на тестовой выборке:', np.round(f1_score(test['toxic'], pred), 4))

F1 LGBMClassifier c подобранными гиперпараметрами на тестовой выборке: 0.7732


### Сравним модель с константной

In [48]:
dm = DummyClassifier(random_state=22, strategy='stratified').fit(features_train, train['toxic'])

In [49]:
dm_pred_v = dm.predict(features_valid)

In [50]:
dm_pred_v_f1 = f1_score(valid['toxic'], dm_pred_v)

In [51]:
dm_pred_t = dm.predict(features_test)

In [52]:
dm_pred_t_f1 = f1_score(test['toxic'], dm_pred_t)

In [53]:
print('F1 Константной модели на валидационной выборке:', np.round(f1_score(valid['toxic'], dm_pred_v), 4))
print(' ')
print('F1 Константной модели на тестовой выборке:', np.round(f1_score(test['toxic'], dm_pred_t), 4))

F1 Константной модели на валидационной выборке: 0.0853
 
F1 Константной модели на тестовой выборке: 0.1014


**Вывод**

1. Нам нужно было построить модель, которая будет классифицировать комментарии на позитивные и негативные
2. На старте у нас был датасет, в котором было 160 тыс. строк и 3 столбца. Один столбец мы удалили, т к он не нёс никакой информации. Обнаружили, что у нас дисбалас классов с разницей в 10 раз.
3. Произвели предобрабтку текстовой информации, чтобы данные можно было подавать в модель.
4. Были обучены разные модели. По результатам лучше всего себя показал LGBMClassifier. Для него были подобраны гиперпараметры и произведена проверка на тесте, f1 вышла 0.78, что удовлетворяет наш критерий.
5. Значение метрики RMSE DummyClassifier 0.10, это говорит нам о том, что наша обученная модель адекватная

## (Дополнительно) Сделаем выборку сбалансированнее и посмотрим что будет

**Увеличиваем число "токсичных" в 5 раз** 

### Напишем функцию для увеличения числа токсичных комментариев

In [54]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=22)
    
    return features_upsampled, target_upsampled

### Получим наши фичи и целевой признак

In [55]:
features_upsampled, target_upsampled = upsample(train['text'], train['toxic'], 5)

### Проверим баланс

In [56]:
target_upsampled.value_counts()

0    100104
1     57000
Name: toxic, dtype: int64

### Готовим данные для модели

In [57]:
vectoriz = TfidfVectorizer(tokenizer=lambda x: change_word(clear_text(x, remove_stop_words=True)))

In [58]:
%%time
X_train = vectoriz.fit_transform(features_upsampled)
X_valid = vectoriz.transform(valid['text'])
X_test = vectoriz.transform(test['text'])

CPU times: total: 1min 35s
Wall time: 1min 35s


### LogisticRegression

In [59]:
lr_up = LogisticRegression(solver='lbfgs', random_state=22).fit(X_train, target_upsampled)

In [60]:
lr_up_pred = lr_up.predict(X_valid)

In [61]:
lr_up_f1 = f1_score(valid['toxic'], lr_up_pred)

In [62]:
lr_up_f1

0.7736226091921211

In [63]:
print(classification_report(valid['toxic'], lr_up_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97     30553
           1       0.75      0.80      0.77      3376

    accuracy                           0.95     33929
   macro avg       0.86      0.89      0.87     33929
weighted avg       0.95      0.95      0.95     33929



***Чуть поднялась f1, после снижения дисбаланса***

In [64]:
def objective_up(trial):
    classifier_up = Pipeline(steps=[("classifier",LGBMClassifier(random_state=22))])

    params = {
        'classifier__lambda_l1': trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        'classifier__lambda_l2': trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        'classifier__num_leaves': trial.suggest_int("num_leaves", 2, 256),
        'classifier__feature_fraction': trial.suggest_float("feature_fraction", 0.4, 1.0),
        'classifier__bagging_fraction': trial.suggest_float("bagging_fraction", 0.4, 1.0),
        'classifier__bagging_freq': trial.suggest_int("bagging_freq", 1, 7),
        'classifier__min_child_samples': trial.suggest_int("min_child_samples", 5, 100)
    }

    classifier_up.set_params(**params)

    classifier_up.fit(X_train, target_upsampled)

    f1_up = cross_val_score(classifier_up, X_valid, valid['toxic'], cv=5, scoring='f1')

    return  f1_up.mean()

In [65]:
%%time
study = optuna.create_study(direction='maximize')
study.optimize(objective_up, n_trials=3)

[I 2022-12-20 21:15:36,977] A new study created in memory with name: no-name-e5c2523b-1701-4919-a537-6833a6a7936d


[LightGBM] [Warning] feature_fraction is set=0.4576915231589659, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4576915231589659
[LightGBM] [Warning] bagging_fraction is set=0.8018869667515953, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8018869667515953
[LightGBM] [Warning] lambda_l1 is set=0.13575868964275786, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.13575868964275786
[LightGBM] [Warning] lambda_l2 is set=0.020382338887404823, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.020382338887404823
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.4576915231589659, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4576915231589659
[LightGBM] [Warning] bagging_fraction is set=0.8018869667515953, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8018869667515953
[LightGBM] [Warni

[I 2022-12-20 21:17:53,977] Trial 0 finished with value: 0.7428887227225556 and parameters: {'lambda_l1': 0.13575868964275786, 'lambda_l2': 0.020382338887404823, 'num_leaves': 73, 'feature_fraction': 0.4576915231589659, 'bagging_fraction': 0.8018869667515953, 'bagging_freq': 7, 'min_child_samples': 7}. Best is trial 0 with value: 0.7428887227225556.


[LightGBM] [Warning] feature_fraction is set=0.9071263563803612, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9071263563803612
[LightGBM] [Warning] bagging_fraction is set=0.9917022474975702, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9917022474975702
[LightGBM] [Warning] lambda_l1 is set=0.837652123943285, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.837652123943285
[LightGBM] [Warning] lambda_l2 is set=2.0095490458985948e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.0095490458985948e-08
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9071263563803612, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9071263563803612
[LightGBM] [Warning] bagging_fraction is set=0.9917022474975702, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9917022474975702
[LightGBM] [Warni

[I 2022-12-20 21:19:46,114] Trial 1 finished with value: 0.6787032573236719 and parameters: {'lambda_l1': 0.837652123943285, 'lambda_l2': 2.0095490458985948e-08, 'num_leaves': 120, 'feature_fraction': 0.9071263563803612, 'bagging_fraction': 0.9917022474975702, 'bagging_freq': 5, 'min_child_samples': 75}. Best is trial 0 with value: 0.7428887227225556.


[LightGBM] [Warning] feature_fraction is set=0.5676248069127506, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5676248069127506
[LightGBM] [Warning] bagging_fraction is set=0.44842668864085605, subsample=1.0 will be ignored. Current value: bagging_fraction=0.44842668864085605
[LightGBM] [Warning] lambda_l1 is set=2.3731730354514595, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3731730354514595
[LightGBM] [Warning] lambda_l2 is set=0.0008239018292013082, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0008239018292013082
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.5676248069127506, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5676248069127506
[LightGBM] [Warning] bagging_fraction is set=0.44842668864085605, subsample=1.0 will be ignored. Current value: bagging_fraction=0.44842668864085605
[LightGBM] [W

[I 2022-12-20 21:21:02,771] Trial 2 finished with value: 0.6925469582133431 and parameters: {'lambda_l1': 2.3731730354514595, 'lambda_l2': 0.0008239018292013082, 'num_leaves': 123, 'feature_fraction': 0.5676248069127506, 'bagging_fraction': 0.44842668864085605, 'bagging_freq': 5, 'min_child_samples': 37}. Best is trial 0 with value: 0.7428887227225556.


CPU times: total: 17min 18s
Wall time: 5min 25s


In [66]:
print(study.best_trial.params)
print(study.best_value)

{'lambda_l1': 0.13575868964275786, 'lambda_l2': 0.020382338887404823, 'num_leaves': 73, 'feature_fraction': 0.4576915231589659, 'bagging_fraction': 0.8018869667515953, 'bagging_freq': 7, 'min_child_samples': 7}
0.7428887227225556


In [67]:
model = LGBMClassifier(**study.best_params, random_state=22).fit(X_train, target_upsampled)

[LightGBM] [Warning] feature_fraction is set=0.4576915231589659, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4576915231589659
[LightGBM] [Warning] bagging_fraction is set=0.8018869667515953, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8018869667515953
[LightGBM] [Warning] lambda_l1 is set=0.13575868964275786, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.13575868964275786
[LightGBM] [Warning] lambda_l2 is set=0.020382338887404823, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.020382338887404823
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7


In [68]:
pred_up_v = model.predict(X_valid)

In [69]:
print('F1 на валидационной выборке:', np.round(f1_score(valid['toxic'], pred_up_v), 4))

F1 на валидационной выборке: 0.7762


In [70]:
pred_up_t = model.predict(X_test)

In [71]:
print('F1 на тестовой выборке:', np.round(f1_score(test['toxic'], pred_up_t), 4))

F1 на тестовой выборке: 0.7845


**Ну а для бустинга ничего не поменялось**